# Noise on spacecrafts

Before, we'll start. The imports and some preparation.

In [1]:
from gravann.util import get_target_point_sampler
from gravann.labels import binder as label_binder
from gravann.functions import unit_conversion as unit_converter
from gravann.util.constants import BODY_SEMI_MAJOR_AXIS, SPEED_OF_LIGHT, SOLAR_CONSTANT

import numpy as np
import torch

# churyumov-gerasimenko, eros, bennu ...
SAMPLE = "churyumov-gerasimenko"

acc_fn = label_binder.bind_label("polyhedral", sample=SAMPLE)

This notebook explores the effects of solar radiation pressure (SRP) on the acceleration of a spacecraft. Further, it calculates a suitable Gaussian noise representing distributions of an accelerometers.

## Gaussian Noise - Gradiometer Inprecision

Let's first use inaccuracy. The [GOCE](https://en.wikipedia.org/wiki/Gravity_Field_and_Steady-State_Ocean_Circulation_Explorer#cite_note-home-7) mission used a "highly sensitive gravity gradiometer consisting of three pairs of accelerometers which measured gravitational gradients along three orthogonal axes." It was designed to determine gravity-field anomalies with a precision of up to
$$
10^{-5} \frac{m}{s^2} = 1 mGal
$$

If we have a look at our two examples Eros and Churyumov–Gerasimenko, we'll have the following conversion factors and acceleration magnitudes.

Let's start with some reasonable altitudes (from the surface of course):

In [2]:
altitudes_norm = np.array([0.1, 0.5, 1.0, 5.0])
altitudes_real = unit_converter.convert_altitude(SAMPLE, altitudes_norm) / 1000.0

points = []
for alt in altitudes_norm:
    target_sampler = get_target_point_sampler(
        N=1, method="altitude", bounds=[alt],
        limit_shape_to_asteroid=f"./3dmeshes/{SAMPLE}.pk", seed=42
    )
    p = target_sampler()
    points.append(p)
points = torch.concatenate(points)
print(points)

tensor([[-0.5028,  0.6385,  0.1428],
        [-0.3490,  1.2492,  0.2721],
        [-0.7257,  1.6392,  0.3664],
        [-0.4965,  1.5836, -5.2992]], dtype=torch.float64)


Now, we calculate the acceleration at these points:

In [3]:
acc_norm = acc_fn(points)
acc_real = unit_converter.convert_acceleration(SAMPLE, acc_norm)

In [4]:
print(f"-----------NORMALIZED-----------")
print(f"The acceleration at normalized altitudes {altitudes_norm}:")
print(acc_norm)
print(f"-----------REAL-----------")
print(f"The acceleration at real altitudes {altitudes_real} [km] in [m/s^2]:")
print(acc_real)

-----------NORMALIZED-----------
The acceleration at normalized altitudes [0.1 0.5 1.  5. ]:
tensor([[ 1.5940, -1.0660, -0.5360],
        [ 0.1861, -0.6291, -0.1557],
        [ 0.1342, -0.2840, -0.0687],
        [ 0.0029, -0.0091,  0.0309]])
-----------REAL-----------
The acceleration at real altitudes [ 0.31266064  1.56330322  3.12660645 15.63303223] [km] in [m/s^2]:
tensor([[ 1.0863e-04, -7.2650e-05, -3.6532e-05],
        [ 1.2682e-05, -4.2875e-05, -1.0610e-05],
        [ 9.1477e-06, -1.9356e-05, -4.6852e-06],
        [ 1.9680e-07, -6.2271e-07,  2.1026e-06]])


As we can see, the acceleration from 1.5 kilometers onward are the in the order of magnitude of the gradiometer's precision. So assuming a certain deviation is reasonable. We model this by adding gaussian noise to the calculated acceleration label for training simulating in-precisions.

But what are reasonable Gaussain parameters?

If we have a look at the values above we can simply "calculate backwards".

In [12]:
gaussian_std_real = torch.tensor(1e-5)
gaussian_std_normal = unit_converter.convert_acceleration(SAMPLE, gaussian_std_real, False)
print(f"Standard deviation for training: {gaussian_std_normal}")

Standard deviation for training: 0.1467314213514328


## Excurs - Transform accelereation to normalized units

$$
10^{-5} \frac{m}{s^2} = 1 \; mGal
$$ should be transformed to normalized units for `churyumov-gerasimenko`. We can do this by using the following formula:

Mass:
$$9.982 \cdot 10^{12} \; kg$$
1 Unit:
$$3126.6064453124995 \; m$$
Gravity Constant:
$$6.67430 \cdot 10^{-11} \; \frac{m^3}{kg \cdot s^2}$$

==> Conversion Factor from unitless to real:
$$
m \cdot G \; / \; unit2metric^{2} = 9.982 \cdot 10^{12} \cdot 6.67430 \cdot 10^{-11} / 3126.6^2 = 0.000068151724 \\

kg \cdot \frac{m^3}{kg \; s^2} / m^2 = \frac{kg \; m^3 }{kg \; s^2 \; m^2} = \frac{m}{s^2}

$$

==> Transformation from real to unitless:
$$
\frac{10^{-5} \; \frac{m}{s^2}}{0.000068151724 \; \frac{m}{s^2}} = 0.1467314311814034227512718533723372867280657492978460823676302011...
$$

## Constant Bias - Solar Radiation Pressure

This probably a bit simplified, but assuming our spacecraft is always facing in the same way to the sun with the maximal cross-section and the asteroid does not reflect a lot. If so, we assume that just one cartesian vector component is affected by the Bias.
We calculate the solar radiation pressure $P$, as shown below:

$$
P = \frac{G_{SC}}{c \cdot R^2}
$$

with the solar constant $G_{SC}$, the speed of light $c$, the distance in astronomical units $R$.
In the following:
$$
S_0 = 1361 \; \frac{W}{m^2} \\
c = 299\;792\;458 \; \frac{m}{s} \\
1 \; AU = 149\,597\,870\,700\; m
$$

Reference: [Solar constant 2015](https://de.wikipedia.org/wiki/Solarkonstante)

With this knowledge. We can calculate the pressure for an arbitrary distance in $Pa = \frac{N}{m^2}$:

In [2]:
solar_radiation_pressure = SOLAR_CONSTANT / (SPEED_OF_LIGHT * (BODY_SEMI_MAJOR_AXIS[SAMPLE] ** 2))
# if fully reflective
# solar_radiation_pressure *= 2.0
print(f"Solar radiation pressure: {solar_radiation_pressure * 1e6} micro Pa")

Solar radiation pressure: 0.37860173912525674 micro Pa


Assuming spacecraft [Rosetta](https://www.esa.int/Enabling_Support/Operations/Rosetta):
The main box 2.8 x 2.1 x 2.0 metres in size. Each "wing": 32 $m^2$.

Mass: 1230 kg (Orbiter Dry Mass), 165 kg (Orbiter Payload), 27 kg (Lander)
(We neglect propellant for since there is no "quick" data, lauch mass was 3000 kg)

Alternatively, we assume a cubesat with the edge lengths of 11 cm and a weight of 0.4 kg.

The solar panels are probably facing the sun. So, we have a maximal size of

In [5]:
max_cross_section = 2.8 * 2.1 + 2 * 32
max_cross_section = 0.11 * 0.11
print(f"Max cross-section: {max_cross_section} m^2")
# mass = 1230.0 + 165.0 + 27.0
mass = 0.5
print(f"Mass: {mass} kg")

Max cross-section: 0.0121 m^2
Mass: 0.5 kg


Ergo, the force on the spacecraft is $F = m \cdot a <==> a = F/m <==> a = (P * A) / m$

In [6]:
solar_acc_real = (solar_radiation_pressure * max_cross_section) / mass
print(f"The acceleration affecting the spacecraft: {solar_acc_real} m/s^2")

The acceleration affecting the spacecraft: 9.162162086831213e-09 m/s^2


Suitable constant bias by calculating backwards:

In [7]:
solar_acc_normalized = unit_converter.convert_acceleration(SAMPLE, torch.tensor(solar_acc_real), False)
print(f"Constant bias: {solar_acc_normalized}")

# Rosetta: 0.000272
# CubeSat: 0.000134

Constant bias: 0.00013443770876619965


## Proposed next steps

Train with the following parameters:
```
# Base: To get solid results and decrease std
SEED = [3, 7, 9, 37, 42, 67, 99, 433, 1999, 2023, ...]
# RQ 3: Effects of (realistic amounts of) noise?
NOISE = [
    GAUSSIAN with mean = 0.0, std = see above --> x2 assume twice in-precision?, sore more for robustness test?
    CONSTANT BIAS with [see above, 0.0, 0.0] --> x2 for fully reflective case?, some more for robustness test?
    COMBINE both types?
    NONE
]
BODIES = [
    churyumov-gerasimenko
    eros
]
# RQ 1: Obtainable precision and its dependence on the ground truth?
METHODS = [
    polyhedral
    mascon
]
# RQ 2: Needed for how far/ close does the sampling needs to be?
SAMPLING_DISTANCE = [
    [0.0, 1.0]
    [1.0, 3.0]
    [3.0, 5.0]  # 5.0 corresponds to ~ 15 km for churyumov-gerasimenko
    # [5.0, 10.0] # NEW? Rosetta's periapsis was 30 km (during normal op), always bad results, not useful for this range, spherical harmonics here
]
# RQ 4: Case study
SAMPLING_METHOD = [
    spherical
    propagated orbit --> one special case for case study with COMBINED noise? Model rosetta's approach?
]

# RQ 1 + 2 + 3 + 4 Follow-up? Applicability with pre-trained model?
1) Train on low-fidelity mesh for 10 000 iterations with every allowed distance
2) Simulate flight/ Only train with high fidelity model with "a lot of" distance for less iterations

```